# 1. Import and Install Dependencies

In [2]:
!pip install tensorflow opencv-python mediapipe

   ---------------------------------------- 0.0/14.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/14.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/14.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/14.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/14.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/14.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/14.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/14.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/14.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/14.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/14.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/14.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/14.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/14.6 MB ? eta -:--:--
   -----------------

In [1]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

# 2. Keypoints using MP Holistic

In [6]:
import cv2
import mediapipe as mp
import numpy as np
from collections import deque

# === Mediapipe (Hands فقط) ===
mp_hands   = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_styles  = mp.solutions.drawing_styles

# ملاحظة: للفيديو نستخدم static_image_mode=False لتتبّع أفضل
hands = mp_hands.Hands(
    static_image_mode=False,
    max_num_hands=1,                 # ثبّت يد واحدة لتثبيت طول المتجه = 42
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)

def mediapipe_detection(image, model):
    """تحويل BGR→RGB، تشغيل الموديل، ثم RGB→BGR (نفس توقيعك)."""
    imgRGB = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    imgRGB.flags.writeable = False
    results = model.process(imgRGB)          # results.multi_hand_landmarks
    imgRGB.flags.writeable = True
    image = cv2.cvtColor(imgRGB, cv2.COLOR_RGB2BGR)
    return image, results

def draw_styled_landmarks(image, results):
    """رسم اليد/الأيدي فقط."""
    if results.multi_hand_landmarks:
        for hand_lms in results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(
                image,
                hand_lms,
                mp_hands.HAND_CONNECTIONS,
                mp_styles.get_default_hand_landmarks_style(),
                mp_styles.get_default_hand_connections_style()
            )

def extract_hand_feats_42(image, results=None):
    """
    يُرجع متجه ميزات بطول 42: (x,y) مطبّعة داخل صندوق اليد،
    مع صندوق الرسم bbox (x1,y1,x2,y2). يُرجع (None,None) إذا لا توجد يد.
    """
    H, W, _ = image.shape
    if results is None:
        _, results = mediapipe_detection(image, hands)

    if not results.multi_hand_landmarks:
        return None, None

    hand = results.multi_hand_landmarks[0]   # يد واحدة فقط (لتثبيت الطول)
    xs = [lm.x for lm in hand.landmark]
    ys = [lm.y for lm in hand.landmark]

    xmin, xmax = min(xs), max(xs)
    ymin, ymax = min(ys), max(ys)
    w = max(xmax - xmin, 1e-6)
    h = max(ymax - ymin, 1e-6)

    feats = []
    for lm in hand.landmark:
        nx = (lm.x - xmin) / w
        ny = (lm.y - ymin) / h
        feats.extend([nx, ny])               # 42 قيمة

    bbox = (int(xmin * W) - 10, int(ymin * H) - 10,
            int(xmax * W) + 10, int(ymax * H) + 10)

    return np.array(feats, dtype=np.float32), bbox

# ----------------------------
# حلقة العرض (وتجهيز نافذة LSTM إن لزم)
# ----------------------------
cap = cv2.VideoCapture(0)   # جرّب 1 لو عندك كاميرا ثانية

# إعداد نافذة انزلاقية للـ LSTM (اختياري)
T = 30                      # طول التسلسل للـ LSTM
window = deque(maxlen=T)

# (اختياري) لو عندك موديل LSTM محفوظ:
# import tensorflow as tf
# lstm = tf.keras.models.load_model('hands_lstm.h5')
# import pickle; meta = pickle.load(open('hands_lstm_labels.pkl','rb'))
# CLASSES = meta['classes']; T = meta['T']  # تأكد أن T يطابق T هنا

if not cap.isOpened():
    raise RuntimeError("لم يتم فتح الكاميرا")

while True:
    ok, frame = cap.read()
    if not ok:
        break

    image, results = mediapipe_detection(frame, hands)
    draw_styled_landmarks(image, results)

    feats42, bbox = extract_hand_feats_42(image, results)
    if feats42 is not None:
        window.append(feats42)

        # رسم صندوق حول اليد
        if bbox is not None:
            x1, y1, x2, y2 = bbox
            cv2.rectangle(image, (x1, y1), (x2, y2), (0, 0, 0), 2)

        # (اختياري) تنبؤ LSTM عندما تمتلئ النافذة:
        # if len(window) == T:
        #     X = np.expand_dims(np.array(window, dtype=np.float32), axis=0)  # (1,T,42)
        #     probs = lstm.predict(X, verbose=0)[0]
        #     idx   = int(np.argmax(probs))
        #     label = CLASSES[idx]; conf = float(probs[idx])
        #     cv2.putText(image, f'{label} ({conf:.2f})', (x1, y1-12),
        #                 cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0,0,0), 2, cv2.LINE_AA)

    cv2.imshow("Hands only (MP 0.10.x)", image)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
hands.close()
cv2.destroyAllWindows()


In [7]:
plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

# 3. Extract Keypoint Values

In [9]:
# def extract_keypoints(results):
#     pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
#     face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
#     lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
#     rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
#     return np.concatenate([pose, face, lh, rh])
def extract_keypoints(results):
    # في حالة عدم وجود أي يد
    if not results.multi_hand_landmarks:
        return np.zeros(42)  # 21 نقطة × (x,y)

    # استخدم أول يد فقط لتثبيت الطول
    hand = results.multi_hand_landmarks[0]
    xs = [lm.x for lm in hand.landmark]
    ys = [lm.y for lm in hand.landmark]

    xmin, xmax = min(xs), max(xs)
    ymin, ymax = min(ys), max(ys)
    w = max(xmax - xmin, 1e-6)
    h = max(ymax - ymin, 1e-6)

    # تطبيع الإحداثيات داخل صندوق اليد
    normalized = []
    for lm in hand.landmark:
        normalized.extend([(lm.x - xmin) / w, (lm.y - ymin) / h])  # 42 قيمة

    return np.array(normalized, dtype=np.float32)


In [10]:
result_test = extract_keypoints(results)
np.save('0', result_test)
# ثم
np.load('0.npy')


array([0.48308682, 1.        , 0.16169877, 0.8559414 , 0.        ,
       0.64902467, 0.09527294, 0.47133756, 0.30716473, 0.40065444,
       0.25014096, 0.5466075 , 0.269723  , 0.265844  , 0.29904374,
       0.1150559 , 0.32028484, 0.        , 0.5454413 , 0.5593933 ,
       0.58153987, 0.38078073, 0.50831   , 0.45291868, 0.4546427 ,
       0.5313354 , 0.7991856 , 0.60286355, 0.7644272 , 0.45490515,
       0.70747817, 0.505908  , 0.6680333 , 0.5774868 , 1.        ,
       0.6506623 , 0.92094773, 0.52188087, 0.86566526, 0.57299083,
       0.84755546, 0.6383981 ], dtype=float32)

# 4. Setup Folders for Collection

In [12]:
# # Path for exported data, numpy arrays
# DATA_PATH = os.path.join('MP_Data') 

# # Actions that we try to detect
# actions = np.array(['2', '7'])

# # Thirty videos worth of data
# no_sequences = 30

# # Videos are going to be 30 frames in length
# sequence_length = 30
# ============================
# Path for exported data
# ============================
DATA_PATH = os.path.join('MP_Data', 'Alphabets')

# ============================
# Arabic Alphabet Actions
# ============================
actions = np.array([
    'ا','ب','ت','ث','ج','ح','خ','د','ذ','ر','ز',
    'س','ش','ص','ض','ط','ظ','ع','غ','ف','ق','ك',
    'ل','م','ن','هـ','و','ي'
])

# ============================
# Number of sequences per letter
# ============================
no_sequences = 100     # عدد الفيديوهات لكل حرف
sequence_length = 50  # عدد الفريمات بكل فيديو

# ============================
# Create folders for dataset
# ============================
for action in actions:
    for sequence in range(no_sequences):
        try:
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass


In [31]:
# hello
## 0
## 1
## 2
## ...
## 29
# thanks

# I love you

In [ ]:
# for action in actions: 
#     for sequence in range(no_sequences):
#         try: 
#             os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
#         except:
#             pass

تم إنشاء جميع المجلدات داخل MP_Data/alphabet بنجاح ✔️


# 5. Collect Keypoint Values for Training and Testing

In [ ]:
# TARGET_FPS = 30
# SECONDS_PER_SEQUENCE = 3           # زوّدها إذا بدك مدة أطول (مثلاً 5)
# sequence_length = int(TARGET_FPS * SECONDS_PER_SEQUENCE)

# cap = cv2.VideoCapture(0)

# with mp_hands.Hands(
#     static_image_mode=False,
#     max_num_hands=1,
#     min_detection_confidence=0.5,
#     min_tracking_confidence=0.5
# ) as hands:
#     for action in actions:
#         for sequence in range(no_sequences):
#             for frame_num in range(sequence_length):
#                 ret, frame = cap.read()
#                 if not ret:
#                     continue

#                 image, results = mediapipe_detection(frame, hands)
#                 draw_styled_landmarks(image, results)

#                 if frame_num == 0:
#                     cv2.putText(image, 'STARTING COLLECTION', (120,200),
#                                 cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 4, cv2.LINE_AA)
#                     cv2.putText(image, f'Collecting frames for {action} Video Number {sequence}', (15,12),
#                                 cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,255), 1, cv2.LINE_AA)
#                     cv2.imshow('OpenCV Feed', image)
#                     cv2.waitKey(3000)  # استعداد 3 ثواني
#                 else:
#                     cv2.putText(image, f'Collecting frames for {action} Video Number {sequence}', (15,12),
#                                 cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,255), 1, cv2.LINE_AA)
#                     cv2.imshow('OpenCV Feed', image)

#                 keypoints = extract_keypoints(results)
#                 npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
#                 np.save(npy_path, keypoints)

#                 if cv2.waitKey(int(1000 / TARGET_FPS)) & 0xFF == ord('q'):
#                     break

# cap.release()
# cv2.destroyAllWindows()
TARGET_FPS = 30
SECONDS_PER_SEQUENCE = 3
sequence_length = int(TARGET_FPS * SECONDS_PER_SEQUENCE)

cap = cv2.VideoCapture(0)

with mp_hands.Hands(
    static_image_mode=False,
    max_num_hands=1,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
) as hands:
    for action in actions:
        for sequence in range(no_sequences):

            print(f"\nجاهزة لتصوير الحرف: {action} | الفيديو رقم {sequence}")
            print("اضغط ENTER للبدء، SPACE لإيقاف مؤقت، ENTER للمتابعة، Q للخروج.")

            # ننتظر دخول المستخدم
            while True:
                key = cv2.waitKey(1) & 0xFF
                if key == 13:    # ENTER
                    print("➡ بدأ التصوير...")
                    break
                if key == ord('q'):
                    cap.release()
                    cv2.destroyAllWindows()
                    raise SystemExit("تم الإنهاء بناءً على طلبك.")

            frame_num = 0
            paused = False

            while frame_num < sequence_length:
                if paused:
                    key = cv2.waitKey(1) & 0xFF
                    if key == 13:  # ENTER → رجوع
                        paused = False
                        print("▶️ تمت المتابعة")
                    elif key == ord('q'):
                        cap.release()
                        cv2.destroyAllWindows()
                        raise SystemExit("تم الإنهاء.")
                    continue

                ret, frame = cap.read()
                if not ret:
                    continue

                image, results = mediapipe_detection(frame, hands)
                draw_styled_landmarks(image, results)

                cv2.putText(
                    image,
                    f'{action} | Video {sequence} | Frame {frame_num}/{sequence_length}',
                    (10, 20),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    0.6,
                    (0, 0, 255),
                    2,
                    cv2.LINE_AA
                )

                cv2.imshow('OpenCV Feed', image)

                # حفظ البيانات
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)

                # مفاتيح التحكم
                key = cv2.waitKey(int(1000 / TARGET_FPS)) & 0xFF
                if key == ord('q'):
                    cap.release()
                    cv2.destroyAllWindows()
                    raise SystemExit("تم الإنهاء.")
                elif key == 32:  # space
                    paused = True
                    print("⏸ تم الإيقاف المؤقت... اضغطي ENTER للمتابعة")
                
                frame_num += 1

cap.release()
cv2.destroyAllWindows()


In [15]:
cap.release()
cv2.destroyAllWindows()

# 6. Preprocess Data and Create Labels and Features

In [2]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [3]:
label_map = {label:num for num, label in enumerate(actions)}

NameError: name 'actions' is not defined

In [18]:
label_map

{'2': 0, '7': 1}

In [19]:
sequences, labels = [], []
for action in actions:
    for sequence in range(no_sequences):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [20]:
np.array(sequences).shape

(60, 90, 42)

In [21]:
np.array(labels).shape

(60,)

In [22]:
X = np.array(sequences)

In [23]:
X.shape

(60, 90, 42)

In [24]:
y = to_categorical(labels).astype(int)

In [25]:
y

array([[1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1]])

In [26]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

In [27]:
y_test.shape

(3, 2)

# 7. Build and Train LSTM Neural Network

In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [5]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [6]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,1662)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

NameError: name 'actions' is not defined

In [31]:
res = [.7, 0.2, 0.1]

In [32]:
actions[np.argmax(res)]

'2'

In [33]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [39]:
print(X_train.shape)  # (N, T, 42)
print(y_train.shape)  # (N,) للـ sparse أو (N, C) للـ one-hot


(57, 90, 42)
(57, 2)


In [41]:
# model.summary()
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Input, LSTM, Dropout, Dense
from tensorflow.keras import backend as K

# مهم: نظّف الجلسة لتفادي كاش لشكل قديم
K.clear_session()

# اجلب الأبعاد من الداتا مباشرة (حتى ما نغلط)
TIMESTEPS = X_train.shape[1]   # 90 عندك
FEAT_DIM  = X_train.shape[2]   # 42 عندك
NUM_CLASSES = y_train.shape[1] # 2 عندك

model = Sequential([
    Input(shape=(TIMESTEPS, FEAT_DIM)),     # << هنا التغيير الأساسي
    LSTM(128, return_sequences=True),
    Dropout(0.4),
    LSTM(64, return_sequences=False),
    Dropout(0.3),
    Dense(128, activation='relu'),
    Dropout(0.2),
    Dense(NUM_CLASSES, activation='softmax')
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',   # لأن y_train شكلها (N, C)
              metrics=['accuracy'])

model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 90, 128)           87552     
                                                                 
 dropout (Dropout)           (None, 90, 128)           0         
                                                                 
 lstm_1 (LSTM)               (None, 64)                49408     
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense (Dense)               (None, 128)               8320      
                                                                 
 dropout_2 (Dropout)         (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 2)                 2

In [42]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

early_stop = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)
reduce_lr  = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=8, min_lr=1e-6)

history = model.fit(
    X_train, y_train,
    epochs=500,
    batch_size=32,
    validation_split=0.2,
    callbacks=[tb_callback, early_stop, reduce_lr],
    verbose=1
)


The history saving thread hit an unexpected error (OperationalError('database or disk is full')).History will not be written to the database.
Epoch 1/500
2/2 [==============================] - ETA: 0s - loss: 0.6988 - accuracy: 0.4222WARNING:tensorflow:AutoGraph could not transform <function Model.make_test_function.<locals>.test_function at 0x000001FA8AA76170> and will run it as-is.
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: [Errno 28] No space left on device
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: [Errno 28] No space left on device
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
2/2 [================

# 8. Make Predictions

In [43]:
res = model.predict(X_test)

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: [Errno 28] No space left on device
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: [Errno 28] No space left on device
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
1/1 [==============================] - 1s 633ms/step


# 9. Save Weights

In [45]:
model.save('action.h5')

In [46]:
del model

In [47]:
from tensorflow.keras.models import load_model
model = load_model('action.h5')


# 10. Evaluation using Confusion Matrix and Accuracy

In [48]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [49]:
yhat = model.predict(X_test)

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: [Errno 28] No space left on device
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: [Errno 28] No space left on device
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
1/1 [==============================] - 0s 481ms/step


In [50]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [51]:
multilabel_confusion_matrix(ytrue, yhat)

array([[[1, 0],
        [0, 2]],

       [[2, 0],
        [0, 1]]], dtype=int64)

In [52]:
accuracy_score(ytrue, yhat)

1.0

# 11. Test in Real Time

In [53]:
colors = [(245,117,16), (117,245,16), (16,117,245)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

In [1]:
# 1) إعدادات
SEQ_LEN   = 90          # طول التسلسل كما درّبت الموديل (عدّلها لو درّبت على قيمة أخرى)
sequence  = []
sentence  = []
threshold = 0.8

cap = cv2.VideoCapture(0)

# 2) استخدم Hands بدل Holistic
with mp_hands.Hands(static_image_mode=False,
                    max_num_hands=1,
                    min_detection_confidence=0.5,
                    min_tracking_confidence=0.5) as hands:

    while cap.isOpened():
        # Read feed
        ret, frame = cap.read()
        if not ret:
            continue

        # Make detections
        image, results = mediapipe_detection(frame, hands)
        # print(results)  # اختياري

        # Draw landmarks (يد فقط)
        draw_styled_landmarks(image, results)

        # 3) استخراج السمات وإدارة نافذة التسلسل
        keypoints = extract_keypoints(results)     # يجب أن تُرجع 42 قيمة
        sequence.append(keypoints)
        sequence = sequence[-SEQ_LEN:]             # احتفظ بآخر SEQ_LEN فقط

        # 4) التنبؤ عندما تمتلي النافذة
        if len(sequence) == SEQ_LEN:
            res = model.predict(np.expand_dims(sequence, axis=0), verbose=0)[0]
            pred_label = actions[np.argmax(res)]
            # print(pred_label)  # اختياري

            # 5) منطق الجملة/الاستقرار
            if res[np.argmax(res)] > threshold:
                if len(sentence) == 0 or pred_label != sentence[-1]:
                    sentence.append(pred_label)

            if len(sentence) > 5:
                sentence = sentence[-5:]

            # 6) عرض احتمالات (اختياري إذا prob_viz و colors موجودين)
            if 'prob_viz' in globals():
                image = prob_viz(res, actions, image, colors)

        # 7) شريط النص
        cv2.rectangle(image, (0, 0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()


NameError: name 'cv2' is not defined

In [58]:
# تقييم الموديل على بيانات التدريب
train_loss, train_acc = model.evaluate(X_train, y_train, verbose=0)
print(f"✅ Training Accuracy: {train_acc * 100:.2f}%")

# تقييم الموديل على بيانات الاختبار
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)
print(f"✅ Testing Accuracy: {test_acc * 100:.2f}%")


✅ Training Accuracy: 100.00%
✅ Testing Accuracy: 100.00%
